In [1]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--jars /home/pklemenkov/hsu/lectures/kafka-clients-0.10.0.1.jar,/home/pklemenkov/hsu/lectures/spark-sql-kafka-0-10_2.11-2.3.0.jar pyspark-shell'
os.environ["PYSPARK_PYTHON"]='python3'
os.environ["SPARK_HOME"]='/opt/cloudera/parcels/SPARK2/lib/spark2/'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.6-src.zip'))
os.environ["PYSPARK_PYTHON"] = 'python3'
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0.cloudera2
      /_/

Using Python version 3.4.3 (default, Nov 17 2016 01:08:31)
SparkSession available as 'spark'.


In [3]:
KAFKA_SERVERS = (
    "virtual-node01.bigdatateam.ru:9092",
    "virtual-node02.bigdatateam.ru:9092",
    "virtual-node03.bigdatateam.ru:9092"
)

In [4]:
messages = spark.readStream\
                .format("kafka")\
                .option("kafka.bootstrap.servers", ",".join(KAFKA_SERVERS))\
                .option("subscribe", "test_topic")\
                .load()

In [5]:
messages.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
import pyspark.sql.functions as f

In [7]:
words = messages.select(f.split(messages.value.cast("string"), " ").alias("words"))\
                .select(f.explode("words").alias("word"))

In [8]:
wordCounts = words.groupBy("word").count()

In [9]:
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("wordcount")\
    .trigger(processingTime="5 seconds")\
    .start()

In [ ]:
spark.sql("select * from wordcount").show()

In [34]:
query.isActive

True

In [35]:
query.stop()

In [38]:
spark.stop()